In [1]:
# ! pip install --user librosa

In [2]:
from pathlib import Path
from scipy.io import wavfile
import scipy.signal
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

## Mount Drive

In [3]:
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

ON_COLAB = is_running_on_colab()
ON_COLAB

True

In [4]:
if ON_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive')
  intermediate_folder = Path('/content/gdrive/MyDrive/Colab Notebooks/Speech recognition')
else:
  intermediate_folder = Path('..') / 'data' / 'intermediate'

Mounted at /content/gdrive


## Read data

In [5]:
X_train = np.load(intermediate_folder / 'train_main_1_sec_audio_mfcc.npy').transpose(0, 2, 1)
X_train.shape

(33566, 32, 20)

In [6]:

X_val = np.load(intermediate_folder / 'val_main_1_sec_audio_mfcc.npy').transpose(0, 2, 1)
X_val.shape

(4619, 32, 20)

In [7]:
# X_test = np.load(intermediate_folder / 'test_main_1_sec_audio_mfcc.npy').transpose(0, 2, 1)
# X_test.shape

In [8]:
y_train_labels = pd.read_csv(intermediate_folder / 'train_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_val_labels = pd.read_csv(intermediate_folder / 'val_main_1_sec_labels.csv', header=None, index_col=False)[0]
# y_test_labels = pd.read_csv(intermediate_folder / 'test_main_1_sec_labels.csv', header=None, index_col=False)[0]
y_train_labels.shape, y_val_labels.shape
# y_train_labels.shape, y_val_labels.shape, y_test_labels.shape

((33566,), (4619,))

In [9]:
le = LabelEncoder()
le.fit(y_train_labels)

y_train = le.transform(y_train_labels)
y_val = le.transform(y_val_labels)
# y_test = le.transform(y_test_labels)
y_train.shape, y_val.shape
# y_train.shape, y_val.shape, y_test.shape

((33566,), (4619,))

In [10]:
pd.Series(y_train).value_counts().sort_index()

0     1667
1     1655
2     1696
3     1662
4     1647
5     1683
6     1723
7     1630
8     1668
9     1650
10    1672
11    1687
12    1708
13    1727
14    1715
15    1672
16    1693
17    1591
18    1686
19    1734
dtype: int64

In [16]:
X_val.shape

(4619, 32, 20)

In [17]:
assert np.mean(X_val[:, :, 0]) == np.mean(X_val.transpose(0, 2, 1)[:, 0, :])

## Standardize

In [20]:
MEAN = X_train[:,:,:13].mean()
STD = np.std(X_train[:,:,:13])
MEAN, STD

(52.29702396428691, 173.88891616468194)

In [21]:
X_train = (X_train - MEAN) / STD
X_val = (X_val - MEAN) / STD
# X_test = (X_test - MEAN) / STD
X_train.shape, X_val.shape
# X_train.shape, X_val.shape, X_test.shape

((33566, 32, 20), (4619, 32, 20))

## Functions

In [22]:
def train(model, patience_train, patience_val=None, learning_rate=.001, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, epochs=20, batch_size=128, cont_train=False):
  print(f'{patience_train=}, {patience_val=}, {learning_rate=}, {epochs=}, {batch_size=}, {cont_train=}, {X_train.shape=}, {y_train.shape=}, {X_val.shape=}, {y_val.shape=}')
  model.summary()
  print(f'Input shape {model.input_shape}, output shape {model.output_shape}')
  early_stopping = []
  if patience_train:
    early_stopping.append(EarlyStopping(monitor='loss', patience=patience_train))
  if patience_val:
    early_stopping.append(EarlyStopping(monitor='loss', patience=patience_val))

  if not cont_train:
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping]);

## New simple RNN

In [27]:
model = Sequential([LSTM(32, input_shape=(32, 20)),
                  Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 32)                6784      
                                                                 
 dense_4 (Dense)             (None, 20)                660       
                                                                 
Total params: 7,444
Trainable params: 7,444
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] - 3s 6ms/step - loss: 2.9997 - accuracy: 0.0516 - val_loss: 2.9981 - val_accuracy: 0.0517
Epoch 2/1000
263/263 [======================

In [26]:
model = Sequential([LSTM(64, input_shape=(32, 20)),
                  Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 64)                21760     
                                                                 
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
Total params: 23,060
Trainable params: 23,060
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] - 3s 6ms/step - loss: 2.9997 - accuracy: 0.0489 - val_loss: 2.9967 - val_accuracy: 0.0524
Epoch 2/1000
263/263 [====================

In [25]:
model = Sequential([LSTM(128, input_shape=(32, 20)),
                  Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 128)               76288     
                                                                 
 dense_2 (Dense)             (None, 20)                2580      
                                                                 
Total params: 78,868
Trainable params: 78,868
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] - 4s 6ms/step - loss: 3.0005 - accuracy: 0.0515 - val_loss: 2.9991 - val_accuracy: 0.0539
Epoch 2/1000
263/263 [====================

In [28]:
model = Sequential([LSTM(256, input_shape=(32, 20)),
                  Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 256)               283648    
                                                                 
 dense_5 (Dense)             (None, 20)                5140      
                                                                 
Total params: 288,788
Trainable params: 288,788
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] - 3s 6ms/step - loss: 3.0016 - accuracy: 0.0501 - val_loss: 2.9988 - val_accuracy: 0.0511
Epoch 2/1000
263/263 [==================

## 2 layers

In [29]:
model = Sequential([LSTM(128, input_shape=(32, 20)),
                    Dense(128, activation='sigmoid'),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 128)               76288     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 20)                2580      
                                                                 
Total params: 95,380
Trainable params: 95,380
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] 

In [30]:
model = Sequential([LSTM(64, input_shape=(32, 20)),
                    Dense(64, activation='sigmoid'),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 64)                21760     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 20)                1300      
                                                                 
Total params: 27,220
Trainable params: 27,220
Non-trainable params: 0
_________________________________________________________________
Input shape (None, 32, 20), output shape (None, 20)
Epoch 1/1000
263/263 [==============================] 

## 4 layers

In [32]:
model = Sequential([LSTM(128, input_shape=(32, 20), return_sequences=True),
                    LSTM(128, input_shape=(32, 20)),
                    Dense(128, activation='sigmoid'),
                    Dense(128, activation='sigmoid'),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 128)               16512     
                                                                 
 dense_15 (Dense)            (None, 20)                2580      
                                            

In [33]:
model = Sequential([LSTM(64, input_shape=(32, 20), return_sequences=True),
                    LSTM(64, input_shape=(32, 20)),
                    Dense(64, activation='sigmoid'),
                    Dense(64, activation='sigmoid'),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 64)                4160      
                                                                 
 dense_18 (Dense)            (None, 20)                1300      
                                           

In [34]:
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.0001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=0.0001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 64)                4160      
                                                                 
 dense_18 (Dense)            (None, 20)                1300      
                                          

In [35]:
train(model=model,
      patience_train=5,
      patience_val=None,
      learning_rate=.00005,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=5, patience_val=None, learning_rate=5e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_13 (LSTM)              (None, 64)                33024     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 64)                4160      
                                                                 
 dense_18 (Dense)            (None, 20)                1300      
                                           

In [37]:
model = Sequential([LSTM(128, input_shape=(32, 20), return_sequences=True, dropout=.1),
                    LSTM(128, dropout=.1),
                    Dropout(.1),
                    Dense(128, activation='sigmoid'),
                    Dropout(.1),
                    Dense(128, activation='sigmoid'),
                    Dropout(.1),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                          

In [38]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                          

In [39]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                          

In [40]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.0001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.0001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                         

In [41]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.0005,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.0005, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_17 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               16512     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                         

In [42]:
model = Sequential([LSTM(64, input_shape=(32, 20), return_sequences=True, dropout=.1),
                    LSTM(64, dropout=.1),
                    Dropout(.1),
                    Dense(64, activation='sigmoid'),
                    Dropout(.1),
                    Dense(64, activation='sigmoid'),
                    Dropout(.1),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_19 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                          

In [43]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_19 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                          

In [44]:
train(model=model,
      patience_train=20,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=20, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 32, 64)            21760     
                                                                 
 lstm_19 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_25 (Dense)            (None, 64)                4160      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                          

In [45]:
model = Sequential([LSTM(128, input_shape=(32, 20), dropout=.1),
                    # LSTM(64, dropout=.1),
                    Dropout(.1),
                    Dense(128, activation='sigmoid'),
                    Dropout(.1),
                    # Dense(64, activation='sigmoid'),
                    # Dropout(.1),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 128)               76288     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 128)               16512     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 20)                2580      
                                          

In [46]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 128)               76288     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 128)               16512     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_29 (Dense)            (None, 20)                2580      
                                          

In [48]:
model = Sequential([LSTM(128, input_shape=(32, 20), dropout=.2, return_sequences=True),
                    LSTM(128, dropout=.2),
                    Dropout(.2),
                    Dense(128, activation='sigmoid'),
                    Dropout(.2),
                    Dense(128, activation='sigmoid'),
                    Dropout(.2),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                          

In [49]:
train(model=model,
      patience_train=20,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=64,
      # cont_train=True,
      )

patience_train=20, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=64, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                           

In [50]:
train(model=model,
      patience_train=20,
      patience_val=None,
      learning_rate=.0000001,
      epochs=1000,
      batch_size=64,
      # cont_train=True,
      )

patience_train=20, patience_val=None, learning_rate=1e-07, epochs=1000, batch_size=64, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                           

In [51]:
train(model=model,
      patience_train=20,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=32,
      # cont_train=True,
      )

patience_train=20, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=32, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                           

In [52]:
train(model=model,
      patience_train=20,
      patience_val=None,
      learning_rate=.0001,
      epochs=1000,
      batch_size=16,
      # cont_train=True,
      )

patience_train=20, patience_val=None, learning_rate=0.0001, epochs=1000, batch_size=16, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_23 (LSTM)              (None, 32, 128)           76288     
                                                                 
 lstm_24 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_33 (Dense)            (None, 128)               16512     
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                          

## Bidirectional, more LSTM, only 1 Dense

In [53]:
model = Sequential([Bidirectional(LSTM(128, dropout=.2, return_sequences=True), input_shape=(32, 20)),
                    Bidirectional(LSTM(128, dropout=.2, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.2, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.2, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.2)),
                    Dropout(.2),
                    Dense(128, activation='sigmoid'),
                    Dropout(.2),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 32, 256)          152576    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 32, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 32, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 32, 2

In [55]:
model = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 20)),
                    Bidirectional(LSTM(128, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.2)),
                    # Dropout(.2),
                    Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_9 (Bidirectio  (None, 32, 256)          152576    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_11 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_40 (Dense)            (None, 128)  

KeyboardInterrupt: ignored

In [56]:
model = Sequential([Bidirectional(LSTM(128, dropout=.2, return_sequences=True), input_shape=(32, 20)),
                    Bidirectional(LSTM(128, dropout=.2, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.2)),
                    Dropout(.2),
                    Dense(128, activation='sigmoid'),
                    Dropout(.2),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_12 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_19 (Dropout)        (None, 256)  

KeyboardInterrupt: ignored

In [57]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_12 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_19 (Dropout)        (None, 256)  

In [58]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.0001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=0.0001, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_12 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_14 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_19 (Dropout)        (None, 256) 

In [59]:
model = Sequential([Bidirectional(LSTM(128, dropout=.1, return_sequences=True), input_shape=(32, 20)),
                    Bidirectional(LSTM(128, dropout=.1, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    # Bidirectional(LSTM(128, return_sequences=True)),
                    Bidirectional(LSTM(128, dropout=.1)),
                    Dropout(.1),
                    Dense(128, activation='sigmoid'),
                    Dropout(.1),
                    # Dense(128, activation='sigmoid'),
                    # Dropout(.2),
                    Dense(20, activation='softmax')])
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=128,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=128, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_15 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_16 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_21 (Dropout)        (None, 256)  

In [60]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.000001,
      epochs=1000,
      batch_size=64,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-06, epochs=1000, batch_size=64, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_15 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_16 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_21 (Dropout)        (None, 256)   

In [61]:
train(model=model,
      patience_train=10,
      patience_val=None,
      learning_rate=.00001,
      epochs=1000,
      batch_size=32,
      # cont_train=True,
      )

patience_train=10, patience_val=None, learning_rate=1e-05, epochs=1000, batch_size=32, cont_train=False, X_train.shape=(33566, 32, 20), y_train.shape=(33566,), X_val.shape=(4619, 32, 20), y_val.shape=(4619,)
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_15 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_16 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_17 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dropout_21 (Dropout)        (None, 256)   

## RNN - 20

In [ ]:
lstm = Sequential([LSTM(20, input_shape=(32, 20)),
                  Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dense_1 (Dense)             (None, 20)                420       
                                                                 
Total params: 3,700
Trainable params: 3,700
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
263/263 [==============================] - 12s 45ms/step - loss: 3.0303 - accuracy: 0.0490 - val_loss: 2.9992 - val_accuracy: 0.0459
Epoch 2/20
263/263 [==============================] - 12s 44ms/step - loss: 2.9970 - accuracy: 0.0489 - val_loss: 2.9969 - val_accuracy: 0.0483
Epoch 3/20
263/263 [==============================] - 13s 50ms/step - loss: 2.9965 - accuracy: 0.0488 - val_loss: 2.9972 - val_accuracy: 0.0515
Epoch 4/20
263/263 [==============================] - 11s 43ms/step - loss: 2.9961 - accuracy: 0.0502 - val_loss: 2.9970 - val_accuracy: 0.0513
Epoch 5/20
263/263 [==============================] - 11s 41ms/step - loss: 2.9959 - accuracy: 0.0511 - val_loss: 2.9972 - val_accuracy: 0.0448
Epoch 6/20
263/263 [==============================] - 12s 45ms/step - loss: 2.9956 - accuracy: 0.0485 - val_loss: 2.9968 - val_accuracy: 0.0489
Epoch 7/20
263/263 [==============================] - 11s 44ms/step - loss: 2.9954 - accuracy: 0.0505 - val_loss: 2.9975 - val_accuracy:

## RNN - 50

In [ ]:
lstm = Sequential([LSTM(50, input_shape=(32, 20)),
                  Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 50)                14200     
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
Total params: 15,220
Trainable params: 15,220
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
1049/1049 [==============================] - 45s 43ms/step - loss: 3.0044 - accuracy: 0.0507 - val_loss: 2.9986 - val_accuracy: 0.0487
Epoch 2/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9984 - accuracy: 0.0488 - val_loss: 2.9970 - val_accuracy: 0.0524
Epoch 3/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9981 - accuracy: 0.0518 - val_loss: 2.9995 - val_accuracy: 0.0470
Epoch 4/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9988 - accuracy: 0.0503 - val_loss: 2.9990 - val_accuracy: 0.0489
Epoch 5/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9981 - accuracy: 0.0494 - val_loss: 3.0023 - val_accuracy: 0.0459
Epoch 6/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9977 - accuracy: 0.0510 - val_loss: 2.9984 - val_accuracy: 0.0507
Epoch 7/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9973 - accuracy: 0.0508 - val_loss: 2.9998 -

## RNN - 100

In [ ]:
lstm = Sequential([LSTM(100, input_shape=(32, 20)),
                  Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               48400     
                                                                 
 dense_3 (Dense)             (None, 20)                2020      
                                                                 
Total params: 50,420
Trainable params: 50,420
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
learning_rate = 0.0001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
   1/1049 [..............................] - ETA: 1:19 - loss: 3.1293 - accuracy: 0.0312

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 44s 42ms/step - loss: 3.0057 - accuracy: 0.0495 - val_loss: 2.9993 - val_accuracy: 0.0489
Epoch 2/20
1049/1049 [==============================] - 45s 43ms/step - loss: 3.0005 - accuracy: 0.0503 - val_loss: 2.9993 - val_accuracy: 0.0533
Epoch 3/20
1049/1049 [==============================] - 45s 42ms/step - loss: 3.0015 - accuracy: 0.0512 - val_loss: 2.9990 - val_accuracy: 0.0498
Epoch 4/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9991 - accuracy: 0.0515 - val_loss: 3.0017 - val_accuracy: 0.0489
Epoch 5/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9986 - accuracy: 0.0512 - val_loss: 3.0038 - val_accuracy: 0.0513
Epoch 6/20
1049/1049 [==============================] - 47s 45ms/step - loss: 2.9994 - accuracy: 0.0526 - val_loss: 3.0029 - val_accuracy: 0.0504
Epoch 7/20
1049/1049 [==============================] - 54s 52ms/step - loss: 2.9979 - accuracy: 0.0539 - val_loss: 3.0013 - val_accura

In [ ]:
# learning_rate = 0.0001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/20
1049/1049 [==============================] - 46s 44ms/step - loss: 2.9937 - accuracy: 0.0533 - val_loss: 3.0010 - val_accuracy: 0.0489
Epoch 2/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9904 - accuracy: 0.0558 - val_loss: 2.9991 - val_accuracy: 0.0520
Epoch 3/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9892 - accuracy: 0.0556 - val_loss: 2.9996 - val_accuracy: 0.0524
Epoch 4/20
1049/1049 [==============================] - 48s 45ms/step - loss: 2.9884 - accuracy: 0.0560 - val_loss: 3.0001 - val_accuracy: 0.0489
Epoch 5/20
1049/1049 [==============================] - 44s 42ms/step - loss: 2.9878 - accuracy: 0.0558 - val_loss: 3.0007 - val_accuracy: 0.0580
Epoch 6/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9871 - accuracy: 0.0560 - val_loss: 3.0007 - val_accuracy: 0.0457
Epoch 7/20
1049/1049 [==============================] - 45s 43ms/step - loss: 2.9868 - accuracy: 0.0566 - val_loss: 3.0002 -

## RNN - 200

In [ ]:
lstm = Sequential([LSTM(200, input_shape=(32, 20)),
                  Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 200)               176800    
                                                                 
 dense_4 (Dense)             (None, 20)                4020      
                                                                 
Total params: 180,820
Trainable params: 180,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
learning_rate = 0.0001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.0001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
1049/1049 [==============================] - 48s 46ms/step - loss: 3.0241 - accuracy: 0.0502 - val_loss: 3.0057 - val_accuracy: 0.0535
Epoch 2/20
1049/1049 [==============================] - 48s 45ms/step - loss: 3.0001 - accuracy: 0.0531 - val_loss: 3.0037 - val_accuracy: 0.0517
Epoch 3/20
1049/1049 [==============================] - 47s 45ms/step - loss: 2.9968 - accuracy: 0.0529 - val_loss: 3.0044 - val_accuracy: 0.0500
Epoch 4/20
1049/1049 [==============================] - 51s 49ms/step - loss: 2.9950 - accuracy: 0.0557 - val_loss: 3.0024 - val_accuracy: 0.0504
Epoch 5/20
1049/1049 [==============================] - 51s 49ms/step - loss: 2.9931 - accuracy: 0.0540 - val_loss: 3.0008 - val_accuracy: 0.0543
Epoch 6/20
1049/1049 [==============================] - 47s 45ms/step - loss: 2.9915 - accuracy: 0.0556 - val_loss: 3.0030 - val_accuracy: 0.0470
Epoch 7/20
1049/1049 [==============================] - 50s 48ms/step - loss: 2.9898 - accuracy: 0.0562 - val_loss: 3.0014 -

## RNN - 100 + Fully connected 100

In [ ]:
lstm = Sequential([LSTM(100, input_shape=(32, 20)),
                   Dense(100, activation='sigmoid'),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 100)               48400     
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 20)                2020      
                                                                 
Total params: 60,520
Trainable params: 60,520
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=.001)

In [ ]:
learning_rate = 0.01

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 53s 50ms/step - loss: 3.0080 - accuracy: 0.0514 - val_loss: 3.0075 - val_accuracy: 0.0511
Epoch 2/20
1049/1049 [==============================] - 55s 53ms/step - loss: 3.0048 - accuracy: 0.0513 - val_loss: 3.0020 - val_accuracy: 0.0509
Epoch 3/20
1049/1049 [==============================] - 52s 50ms/step - loss: 3.0031 - accuracy: 0.0519 - val_loss: 2.9989 - val_accuracy: 0.0552
Epoch 4/20
1049/1049 [==============================] - 53s 50ms/step - loss: 3.0007 - accuracy: 0.0511 - val_loss: 2.9992 - val_accuracy: 0.0481
Epoch 5/20
1049/1049 [==============================] - 57s 55ms/step - loss: 2.9987 - accuracy: 0.0506 - val_loss: 2.9993 - val_accuracy: 0.0489
Epoch 6/20
1049/1049 [==============================] - 56s 53ms/step - loss: 2.9972 - accuracy: 0.0511 - val_loss: 2.9993 - val_accuracy: 0.0526
Epoch 7/20
1049/1049 [==============================] - 57s 54ms/step - loss: 2.9963 - accuracy: 0.0536 - val_loss: 2.9986 - val_accura

In [ ]:
# learning_rate = 0.01
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
   1/1049 [..............................] - ETA: 1:05 - loss: 3.0199 - accuracy: 0.0312

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 54s 52ms/step - loss: 2.9983 - accuracy: 0.0503 - val_loss: 3.0000 - val_accuracy: 0.0465
Epoch 2/20
1049/1049 [==============================] - 53s 51ms/step - loss: 2.9981 - accuracy: 0.0474 - val_loss: 2.9965 - val_accuracy: 0.0522
Epoch 3/20
1049/1049 [==============================] - 55s 52ms/step - loss: 2.9976 - accuracy: 0.0502 - val_loss: 2.9993 - val_accuracy: 0.0502
Epoch 4/20
1049/1049 [==============================] - 53s 50ms/step - loss: 2.9980 - accuracy: 0.0483 - val_loss: 2.9970 - val_accuracy: 0.0502
Epoch 5/20
1049/1049 [==============================] - 53s 50ms/step - loss: 2.9976 - accuracy: 0.0510 - val_loss: 2.9976 - val_accuracy: 0.0465
Epoch 6/20
1049/1049 [==============================] - 54s 51ms/step - loss: 2.9973 - accuracy: 0.0532 - val_loss: 3.0005 - val_accuracy: 0.0463
Epoch 7/20
1049/1049 [==============================] - 54s 51ms/step - loss: 2.9976 - accuracy: 0.0473 - val_loss: 2.9997 - val_accura

## RNN - Bidirections, small LSTM, larger Dense

In [ ]:
lstm = Sequential([Bidirectional(LSTM(20), input_shape=(32, 20)),
                   Dense(100, activation='sigmoid'),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 40)               6560      
 l)                                                              
                                                                 
 dense_7 (Dense)             (None, 100)               4100      
                                                                 
 dense_8 (Dense)             (None, 20)                2020      
                                                                 
Total params: 12,680
Trainable params: 12,680
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=.001)

In [ ]:
learning_rate = 0.01

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 78s 74ms/step - loss: 3.0075 - accuracy: 0.0502 - val_loss: 3.0005 - val_accuracy: 0.0509
Epoch 2/20
1049/1049 [==============================] - 78s 75ms/step - loss: 3.0050 - accuracy: 0.0523 - val_loss: 3.0041 - val_accuracy: 0.0546
Epoch 3/20
1049/1049 [==============================] - 79s 75ms/step - loss: 3.0045 - accuracy: 0.0506 - val_loss: 3.0085 - val_accuracy: 0.0470
Epoch 4/20
1049/1049 [==============================] - 79s 75ms/step - loss: 3.0034 - accuracy: 0.0504 - val_loss: 3.0014 - val_accuracy: 0.0526
Epoch 5/20
1049/1049 [==============================] - 78s 74ms/step - loss: 3.0014 - accuracy: 0.0524 - val_loss: 3.0061 - val_accuracy: 0.0520
Epoch 6/20
1049/1049 [==============================] - 77s 74ms/step - loss: 3.0012 - accuracy: 0.0519 - val_loss: 3.0041 - val_accuracy: 0.0485
Epoch 7/20
1049/1049 [==============================] - 77s 74ms/step - loss: 3.0007 - accuracy: 0.0518 - val_loss: 2.9991 - val_accura

In [ ]:
# learning_rate = 0.01
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 76s 73ms/step - loss: 2.9979 - accuracy: 0.0512 - val_loss: 2.9989 - val_accuracy: 0.0487
Epoch 2/20
1049/1049 [==============================] - 79s 75ms/step - loss: 2.9976 - accuracy: 0.0499 - val_loss: 2.9994 - val_accuracy: 0.0522
Epoch 3/20
1049/1049 [==============================] - 80s 76ms/step - loss: 2.9974 - accuracy: 0.0515 - val_loss: 2.9969 - val_accuracy: 0.0522
Epoch 4/20
1049/1049 [==============================] - 81s 77ms/step - loss: 2.9975 - accuracy: 0.0516 - val_loss: 2.9989 - val_accuracy: 0.0487
Epoch 5/20
1049/1049 [==============================] - 79s 75ms/step - loss: 2.9975 - accuracy: 0.0483 - val_loss: 2.9998 - val_accuracy: 0.0520
Epoch 6/20
1049/1049 [==============================] - 78s 74ms/step - loss: 2.9973 - accuracy: 0.0500 - val_loss: 2.9993 - val_accuracy: 0.0463


## RNN - Bidirections, 2 larger LSTMS, larger Dense

In [ ]:
lstm = Sequential([Bidirectional(LSTM(64, return_sequences=True), input_shape=(32, 20)),
                   Bidirectional(LSTM(64)),
                   Dense(100, activation='sigmoid'),
                   Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_1 (Bidirectio  (None, 32, 128)          43520     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_9 (Dense)             (None, 100)               12900     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 20)                2020      
                                                                 
Total params: 157,256
Trainable params: 157,256
Non-tr

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=.001)

In [ ]:
learning_rate = 0.01

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1049/1049 [==============================] - 122s 116ms/step - loss: 3.0087 - accuracy: 0.0497 - val_loss: 2.9960 - val_accuracy: 0.0487
Epoch 2/20
1049/1049 [==============================] - 121s 115ms/step - loss: 2.9963 - accuracy: 0.0484 - val_loss: 2.9957 - val_accuracy: 0.0500
Epoch 3/20
1049/1049 [==============================] - 121s 115ms/step - loss: 2.9960 - accuracy: 0.0494 - val_loss: 2.9957 - val_accuracy: 0.0517
Epoch 4/20
1049/1049 [==============================] - 121s 115ms/step - loss: 2.9959 - accuracy: 0.0513 - val_loss: 2.9958 - val_accuracy: 0.0520
Epoch 5/20
1049/1049 [==============================] - 122s 116ms/step - loss: 2.9959 - accuracy: 0.0498 - val_loss: 2.9960 - val_accuracy: 0.0517
Epoch 6/20
1049/1049 [==============================] - 122s 117ms/step - loss: 2.9958 - accuracy: 0.0504 - val_loss: 2.9962 - val_accuracy: 0.0517
Epoch 7/20
1049/1049 [==============================] - 122s 117ms/step - loss: 2.9960 - accuracy: 0.0509 - val_loss: 2.996

In [ ]:
# learning_rate = 0.01
lstm.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[early_stopping]);

Epoch 1/20
1049/1049 [==============================] - 127s 121ms/step - loss: 2.9984 - accuracy: 0.0516 - val_loss: 2.9966 - val_accuracy: 0.0509
Epoch 2/20
1049/1049 [==============================] - 121s 115ms/step - loss: 2.9978 - accuracy: 0.0501 - val_loss: 2.9972 - val_accuracy: 0.0507
Epoch 3/20
1049/1049 [==============================] - 128s 122ms/step - loss: 2.9983 - accuracy: 0.0481 - val_loss: 2.9971 - val_accuracy: 0.0485
Epoch 4/20
1049/1049 [==============================] - 125s 119ms/step - loss: 2.9977 - accuracy: 0.0495 - val_loss: 2.9983 - val_accuracy: 0.0507
Epoch 5/20
1049/1049 [==============================] - 127s 121ms/step - loss: 2.9981 - accuracy: 0.0501 - val_loss: 2.9974 - val_accuracy: 0.0504
Epoch 6/20
1049/1049 [==============================] - 124s 118ms/step - loss: 2.9982 - accuracy: 0.0500 - val_loss: 2.9974 - val_accuracy: 0.0487


## RNN - Bidirections, 2 larger LSTMS, larger Dense

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 20)),
                  #  Dropout(.5),
                   Bidirectional(LSTM(64)),
                  #  Dropout(.5),
                   Dense(100, activation='sigmoid'),
                   Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_9 (Bidirectio  (None, 32, 256)          152576    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 128)              164352    
 onal)                                                           
                                                                 
 dense_17 (Dense)            (None, 100)               12900     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_18 (Dense)            (None, 20)                2020      
                                                                 
Total params: 331,848
Trainable params: 331,848
Non-t

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 17s 125ms/step - loss: 3.0350 - accuracy: 0.0513 - val_loss: 2.9959 - val_accuracy: 0.0541
Epoch 2/20
132/132 [==============================] - 16s 119ms/step - loss: 2.9974 - accuracy: 0.0501 - val_loss: 2.9960 - val_accuracy: 0.0517
Epoch 3/20
132/132 [==============================] - 16s 119ms/step - loss: 2.9961 - accuracy: 0.0516 - val_loss: 2.9961 - val_accuracy: 0.0517
Epoch 4/20
132/132 [==============================] - 16s 119ms/step - loss: 2.9959 - accuracy: 0.0503 - val_loss: 2.9960 - val_accuracy: 0.0487
Epoch 5/20
132/132 [==============================] - 16s 122ms/step - loss: 2.9960 - accuracy: 0.0487 - val_loss: 2.9959 - val_accuracy: 0.0517
Epoch 6/20
132/132 [==============================] - 16s 122ms/step - loss: 2.9959 - accuracy: 0.0494 - val_loss: 2.9959 - val_accuracy: 0.0517
Epoch 7/20
132/132 [==============================] - 16s 122ms/step - loss: 2.9958 - accuracy: 0.0495 - val_loss: 2.9960 - val_ac

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 16s 120ms/step - loss: 2.9957 - accuracy: 0.0516 - val_loss: 2.9961 - val_accuracy: 0.0522
Epoch 2/20
132/132 [==============================] - 16s 124ms/step - loss: 2.9955 - accuracy: 0.0496 - val_loss: 2.9960 - val_accuracy: 0.0509
Epoch 3/20
132/132 [==============================] - 16s 123ms/step - loss: 2.9957 - accuracy: 0.0509 - val_loss: 2.9962 - val_accuracy: 0.0465
Epoch 4/20
132/132 [==============================] - 16s 119ms/step - loss: 2.9958 - accuracy: 0.0500 - val_loss: 2.9962 - val_accuracy: 0.0517
Epoch 5/20
132/132 [==============================] - 16s 121ms/step - loss: 2.9958 - accuracy: 0.0517 - val_loss: 2.9961 - val_accuracy: 0.0517


## RNN - 3 Bidirections

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.2), input_shape=(32, 20)),
                   Bidirectional(LSTM(128, return_sequences=True, dropout=.2)),
                   Bidirectional(LSTM(128, dropout=.2)),
                  #  Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_11 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_12 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_13 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_19 (Dense)            (None, 20)                5140      
                                                                 
Total params: 946,196
Trainable params: 946,196
Non-trainable params: 0
_______________________________________________

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


132/132 [==============================] - 24s 181ms/step - loss: 3.0053 - accuracy: 0.0513 - val_loss: 3.0008 - val_accuracy: 0.0487
Epoch 2/20
132/132 [==============================] - 24s 179ms/step - loss: 2.9975 - accuracy: 0.0520 - val_loss: 2.9969 - val_accuracy: 0.0528
Epoch 3/20
132/132 [==============================] - 23s 175ms/step - loss: 2.9956 - accuracy: 0.0526 - val_loss: 2.9982 - val_accuracy: 0.0504
Epoch 4/20
132/132 [==============================] - 22s 168ms/step - loss: 2.9951 - accuracy: 0.0531 - val_loss: 2.9968 - val_accuracy: 0.0533
Epoch 5/20
132/132 [==============================] - 23s 171ms/step - loss: 2.9945 - accuracy: 0.0566 - val_loss: 2.9985 - val_accuracy: 0.0459
Epoch 6/20
132/132 [==============================] - 23s 173ms/step - loss: 2.9937 - accuracy: 0.0541 - val_loss: 2.9960 - val_accuracy: 0.0491
Epoch 7/20
132/132 [==============================] - 22s 169ms/step - loss: 2.9937 - accuracy: 0.0548 - val_loss: 2.9979 - val_accuracy: 0.0

## RNN - 3 Bidirections with Dense and Dropout

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.2), input_shape=(32, 20)),
                   Bidirectional(LSTM(128, return_sequences=True, dropout=.2)),
                   Bidirectional(LSTM(128, dropout=.2)),
                   Dense(100, activation='sigmoid'),
                   Dropout(.2),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_17 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_18 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_19 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_22 (Dense)            (None, 100)               25700     
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                     

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 24s 179ms/step - loss: 3.0135 - accuracy: 0.0487 - val_loss: 2.9962 - val_accuracy: 0.0507
Epoch 2/20
132/132 [==============================] - 25s 188ms/step - loss: 2.9970 - accuracy: 0.0482 - val_loss: 2.9959 - val_accuracy: 0.0465
Epoch 3/20
132/132 [==============================] - 24s 180ms/step - loss: 2.9961 - accuracy: 0.0499 - val_loss: 2.9960 - val_accuracy: 0.0517
Epoch 4/20
132/132 [==============================] - 25s 186ms/step - loss: 2.9959 - accuracy: 0.0494 - val_loss: 2.9957 - val_accuracy: 0.0507
Epoch 5/20
132/132 [==============================] - 23s 173ms/step - loss: 2.9959 - accuracy: 0.0491 - val_loss: 2.9958 - val_accuracy: 0.0517
Epoch 6/20
132/132 [==============================] - 24s 180ms/step - loss: 2.9959 - accuracy: 0.0518 - val_loss: 2.9958 - val_accuracy: 0.0517
Epoch 7/20
132/132 [==============================] - 24s 182ms/step - loss: 2.9958 - accuracy: 0.0516 - val_loss: 2.9958 - val_ac

## RNN - 3 Bidirection, no dropout

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 20)),
                   Bidirectional(LSTM(128, return_sequences=True)),
                   Bidirectional(LSTM(128)),
                  #  Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_20 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_21 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_22 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_24 (Dense)            (None, 20)                5140      
                                                                 
Total params: 946,196
Trainable params: 946,196
Non-trainable params: 0
_______________________________________________

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 21s 158ms/step - loss: 3.0055 - accuracy: 0.0486 - val_loss: 3.0007 - val_accuracy: 0.0528
Epoch 2/20
132/132 [==============================] - 21s 159ms/step - loss: 2.9981 - accuracy: 0.0521 - val_loss: 3.0005 - val_accuracy: 0.0517
Epoch 3/20
132/132 [==============================] - 20s 150ms/step - loss: 2.9966 - accuracy: 0.0531 - val_loss: 3.0003 - val_accuracy: 0.0452
Epoch 4/20
132/132 [==============================] - 21s 160ms/step - loss: 2.9953 - accuracy: 0.0521 - val_loss: 2.9981 - val_accuracy: 0.0483
Epoch 5/20
132/132 [==============================] - 20s 154ms/step - loss: 2.9942 - accuracy: 0.0556 - val_loss: 2.9979 - val_accuracy: 0.0489
Epoch 6/20
132/132 [==============================] - 21s 157ms/step - loss: 2.9926 - accuracy: 0.0550 - val_loss: 2.9979 - val_accuracy: 0.0513
Epoch 7/20
132/132 [==============================] - 20s 151ms/step - loss: 2.9916 - accuracy: 0.0561 - val_loss: 2.9973 - val_ac

## RNN - 3 Bidirection, tiny dropout

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.1), input_shape=(32, 20)),
                   Bidirectional(LSTM(128, return_sequences=True, dropout=.1)),
                   Bidirectional(LSTM(128, dropout=.1)),
                  #  Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_23 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_24 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_25 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_25 (Dense)            (None, 20)                5140      
                                                                 
Total params: 946,196
Trainable params: 946,196
Non-trainable params: 0
_______________________________________________

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


132/132 [==============================] - 23s 173ms/step - loss: 3.0047 - accuracy: 0.0506 - val_loss: 2.9986 - val_accuracy: 0.0463
Epoch 2/20
132/132 [==============================] - 22s 166ms/step - loss: 2.9971 - accuracy: 0.0529 - val_loss: 2.9975 - val_accuracy: 0.0550
Epoch 3/20
132/132 [==============================] - 23s 176ms/step - loss: 2.9955 - accuracy: 0.0550 - val_loss: 2.9974 - val_accuracy: 0.0539
Epoch 4/20
132/132 [==============================] - 23s 176ms/step - loss: 2.9951 - accuracy: 0.0545 - val_loss: 2.9974 - val_accuracy: 0.0509
Epoch 5/20
132/132 [==============================] - 24s 186ms/step - loss: 2.9945 - accuracy: 0.0526 - val_loss: 2.9967 - val_accuracy: 0.0548
Epoch 6/20
132/132 [==============================] - 22s 170ms/step - loss: 2.9940 - accuracy: 0.0561 - val_loss: 2.9971 - val_accuracy: 0.0476
Epoch 7/20
132/132 [==============================] - 23s 176ms/step - loss: 2.9929 - accuracy: 0.0575 - val_loss: 2.9980 - val_accuracy: 0.0

## RNN - 3 Bidirection, tiny dropout + Dense

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.1), input_shape=(32, 20)),
                   Bidirectional(LSTM(128, return_sequences=True, dropout=.1)),
                   Bidirectional(LSTM(128, dropout=.1)),
                   Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_26 (Bidirecti  (None, 32, 256)          152576    
 onal)                                                           
                                                                 
 bidirectional_27 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_28 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_26 (Dense)            (None, 100)               25700     
                                                                 
 dense_27 (Dense)            (None, 20)                2020      
                                                     

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 20), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train, y_train, batch_size=256, epochs=20, validation_data=(X_val, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 23s 175ms/step - loss: 3.0060 - accuracy: 0.0509 - val_loss: 3.0009 - val_accuracy: 0.0509
Epoch 2/20
132/132 [==============================] - 23s 173ms/step - loss: 3.0005 - accuracy: 0.0496 - val_loss: 2.9989 - val_accuracy: 0.0500
Epoch 3/20
132/132 [==============================] - 23s 174ms/step - loss: 2.9998 - accuracy: 0.0512 - val_loss: 2.9997 - val_accuracy: 0.0487
Epoch 4/20
132/132 [==============================] - 23s 175ms/step - loss: 3.0001 - accuracy: 0.0538 - val_loss: 3.0008 - val_accuracy: 0.0478
Epoch 5/20
132/132 [==============================] - 23s 171ms/step - loss: 2.9983 - accuracy: 0.0530 - val_loss: 2.9993 - val_accuracy: 0.0465
Epoch 6/20
132/132 [==============================] - 23s 178ms/step - loss: 2.9987 - accuracy: 0.0497 - val_loss: 2.9991 - val_accuracy: 0.0422
Epoch 7/20
132/132 [==============================] - 23s 177ms/step - loss: 2.9989 - accuracy: 0.0502 - val_loss: 2.9981 - val_ac

## RNN - 3 Bidirection, tiny dropout + Dense, only 13 features

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.1), input_shape=(32, 13)),
                   Bidirectional(LSTM(128, return_sequences=True, dropout=.1)),
                   Bidirectional(LSTM(128, dropout=.1)),
                   Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_32 (Bidirecti  (None, 32, 256)          145408    
 onal)                                                           
                                                                 
 bidirectional_33 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_34 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_30 (Dense)            (None, 100)               25700     
                                                                 
 dense_31 (Dense)            (None, 20)                2020      
                                                     

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 13), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 23s 176ms/step - loss: 3.0060 - accuracy: 0.0488 - val_loss: 3.0030 - val_accuracy: 0.0478
Epoch 2/20
132/132 [==============================] - 25s 192ms/step - loss: 3.0002 - accuracy: 0.0515 - val_loss: 3.0011 - val_accuracy: 0.0465
Epoch 3/20
132/132 [==============================] - 24s 179ms/step - loss: 3.0001 - accuracy: 0.0506 - val_loss: 2.9979 - val_accuracy: 0.0522
Epoch 4/20
132/132 [==============================] - 24s 179ms/step - loss: 2.9993 - accuracy: 0.0510 - val_loss: 2.9992 - val_accuracy: 0.0522
Epoch 5/20
132/132 [==============================] - 23s 174ms/step - loss: 2.9996 - accuracy: 0.0494 - val_loss: 2.9986 - val_accuracy: 0.0535
Epoch 6/20
132/132 [==============================] - 24s 185ms/step - loss: 2.9997 - accuracy: 0.0487 - val_loss: 2.9991 - val_accuracy: 0.0487


## RNN - 3 Bidirection, no dropout + Dense, only 13 features

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 13)),
                   Bidirectional(LSTM(128, return_sequences=True)),
                   Bidirectional(LSTM(128)),
                   Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_35 (Bidirecti  (None, 32, 256)          145408    
 onal)                                                           
                                                                 
 bidirectional_36 (Bidirecti  (None, 32, 256)          394240    
 onal)                                                           
                                                                 
 bidirectional_37 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_32 (Dense)            (None, 100)               25700     
                                                                 
 dense_33 (Dense)            (None, 20)                2020      
                                                     

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 13), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 23s 173ms/step - loss: 3.0071 - accuracy: 0.0510 - val_loss: 3.0010 - val_accuracy: 0.0517
Epoch 2/20
132/132 [==============================] - 23s 171ms/step - loss: 3.0009 - accuracy: 0.0497 - val_loss: 3.0023 - val_accuracy: 0.0487
Epoch 3/20
132/132 [==============================] - 22s 163ms/step - loss: 3.0011 - accuracy: 0.0501 - val_loss: 3.0002 - val_accuracy: 0.0487
Epoch 4/20
132/132 [==============================] - 23s 173ms/step - loss: 2.9995 - accuracy: 0.0488 - val_loss: 3.0020 - val_accuracy: 0.0517
Epoch 5/20
132/132 [==============================] - 23s 171ms/step - loss: 2.9992 - accuracy: 0.0510 - val_loss: 3.0020 - val_accuracy: 0.0489
Epoch 6/20
132/132 [==============================] - 22s 165ms/step - loss: 2.9984 - accuracy: 0.0512 - val_loss: 3.0034 - val_accuracy: 0.0507
Epoch 7/20
132/132 [==============================] - 22s 167ms/step - loss: 2.9985 - accuracy: 0.0504 - val_loss: 2.9993 - val_ac

In [ ]:
# learning_rate = 0.002
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

In [ ]:
# learning_rate = 0.002
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 23s 171ms/step - loss: 2.9902 - accuracy: 0.0570 - val_loss: 3.0032 - val_accuracy: 0.0515
Epoch 2/20
132/132 [==============================] - 21s 161ms/step - loss: 2.9899 - accuracy: 0.0605 - val_loss: 3.0038 - val_accuracy: 0.0522
Epoch 3/20
132/132 [==============================] - 21s 163ms/step - loss: 2.9897 - accuracy: 0.0586 - val_loss: 2.9976 - val_accuracy: 0.0487
Epoch 4/20
132/132 [==============================] - 22s 165ms/step - loss: 2.9878 - accuracy: 0.0604 - val_loss: 2.9984 - val_accuracy: 0.0461
Epoch 5/20
132/132 [==============================] - 21s 159ms/step - loss: 2.9866 - accuracy: 0.0603 - val_loss: 3.0041 - val_accuracy: 0.0517
Epoch 6/20
132/132 [==============================] - 21s 162ms/step - loss: 2.9845 - accuracy: 0.0642 - val_loss: 3.0025 - val_accuracy: 0.0533
Epoch 7/20
 93/132 [====================>.........] - ETA: 5s - loss: 2.9827 - accuracy: 0.0635

KeyboardInterrupt: ignored

In [ ]:
learning_rate = 0.003

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
# learning_rate = 0.003
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


132/132 [==============================] - 22s 164ms/step - loss: 2.9852 - accuracy: 0.0604 - val_loss: 2.9999 - val_accuracy: 0.0515
Epoch 2/20
132/132 [==============================] - 23s 173ms/step - loss: 2.9853 - accuracy: 0.0586 - val_loss: 2.9992 - val_accuracy: 0.0513
Epoch 3/20
132/132 [==============================] - 21s 161ms/step - loss: 2.9862 - accuracy: 0.0615 - val_loss: 3.0023 - val_accuracy: 0.0554
Epoch 4/20
132/132 [==============================] - 23s 172ms/step - loss: 2.9829 - accuracy: 0.0614 - val_loss: 3.0048 - val_accuracy: 0.0582
Epoch 5/20
132/132 [==============================] - 21s 158ms/step - loss: 2.9845 - accuracy: 0.0606 - val_loss: 3.0008 - val_accuracy: 0.0539
Epoch 6/20
132/132 [==============================] - 22s 166ms/step - loss: 2.9803 - accuracy: 0.0627 - val_loss: 3.0051 - val_accuracy: 0.0507
Epoch 7/20
132/132 [==============================] - 21s 159ms/step - loss: 2.9795 - accuracy: 0.0623 - val_loss: 3.0124 - val_accuracy: 0.0

## RNN - 2 Bidirection, only 13 features

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 13)),
                   Bidirectional(LSTM(128)),
                  #  Bidirectional(LSTM(128)),
                  #  Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_40 (Bidirecti  (None, 32, 256)          145408    
 onal)                                                           
                                                                 
 bidirectional_41 (Bidirecti  (None, 256)              394240    
 onal)                                                           
                                                                 
 dense_35 (Dense)            (None, 20)                5140      
                                                                 
Total params: 544,788
Trainable params: 544,788
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 13), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 15s 111ms/step - loss: 3.0084 - accuracy: 0.0486 - val_loss: 2.9999 - val_accuracy: 0.0509
Epoch 2/20
132/132 [==============================] - 15s 115ms/step - loss: 2.9982 - accuracy: 0.0530 - val_loss: 3.0005 - val_accuracy: 0.0522
Epoch 3/20
132/132 [==============================] - 15s 114ms/step - loss: 2.9965 - accuracy: 0.0531 - val_loss: 3.0001 - val_accuracy: 0.0522
Epoch 4/20
132/132 [==============================] - 16s 124ms/step - loss: 2.9947 - accuracy: 0.0557 - val_loss: 3.0001 - val_accuracy: 0.0459
Epoch 5/20
132/132 [==============================] - 15s 111ms/step - loss: 2.9931 - accuracy: 0.0573 - val_loss: 3.0033 - val_accuracy: 0.0478
Epoch 6/20
132/132 [==============================] - 15s 114ms/step - loss: 2.9924 - accuracy: 0.0583 - val_loss: 2.9998 - val_accuracy: 0.0507
Epoch 7/20
132/132 [==============================] - 15s 110ms/step - loss: 2.9895 - accuracy: 0.0584 - val_loss: 3.0002 - val_ac

## RNN - 2 Bidirection, only 13 features, a bit of dropout

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True, dropout=.1), input_shape=(32, 13)),
                   Bidirectional(LSTM(128, dropout=.1)),
                  #  Bidirectional(LSTM(128)),
                  #  Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 32, 256)          145408    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 20)                5140      
                                                                 
Total params: 544,788
Trainable params: 544,788
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 13), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.001
lstm.fit(X_train[:,:,:13], y_train, batch_size=256, epochs=20, validation_data=(X_val[:,:,:13], y_val),
         callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


132/132 [==============================] - 166s 1s/step - loss: 3.0078 - accuracy: 0.0525 - val_loss: 3.0002 - val_accuracy: 0.0457
Epoch 2/20
132/132 [==============================] - 166s 1s/step - loss: 2.9989 - accuracy: 0.0525 - val_loss: 3.0012 - val_accuracy: 0.0474
Epoch 3/20
132/132 [==============================] - 168s 1s/step - loss: 2.9972 - accuracy: 0.0544 - val_loss: 3.0005 - val_accuracy: 0.0448
Epoch 4/20
132/132 [==============================] - 165s 1s/step - loss: 2.9958 - accuracy: 0.0558 - val_loss: 3.0000 - val_accuracy: 0.0489
Epoch 5/20
132/132 [==============================] - 168s 1s/step - loss: 2.9947 - accuracy: 0.0552 - val_loss: 3.0000 - val_accuracy: 0.0470
Epoch 6/20
132/132 [==============================] - 165s 1s/step - loss: 2.9943 - accuracy: 0.0531 - val_loss: 2.9981 - val_accuracy: 0.0507
Epoch 7/20
132/132 [==============================] - 166s 1s/step - loss: 2.9932 - accuracy: 0.0577 - val_loss: 2.9983 - val_accuracy: 0.0530
Epoch 8/20

KeyboardInterrupt: ignored

## RNN - 3 Bidirection, no dropout + Dense, only 13 features

In [ ]:
MEAN = X_train[:,:,:13].mean()
STDEV = np.std(X_train[:,:,:13])
MEAN, STDEV

(52.29702396428691, 173.88891616468194)

In [ ]:
lstm = Sequential([Bidirectional(LSTM(128, return_sequences=True), input_shape=(32, 13)),
                   Bidirectional(LSTM(128, return_sequences=True)),
                   Bidirectional(LSTM(128)),
                   Dense(100, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_5 (Bidirectio  (None, 32, 256)          145408    
 nal)                                                            
                                                                 
 bidirectional_6 (Bidirectio  (None, 32, 256)          394240    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               25700     
                                                                 
 dense_4 (Dense)             (None, 20)                2020      
                                                      

In [ ]:
lstm.input_shape, lstm.output_shape

((None, 32, 13), (None, 20))

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=4, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [ ]:
learning_rate = 0.005

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# learning_rate = 0.005
lstm.fit((X_train[:,:,:13] - MEAN) / STDEV, y_train,
         batch_size=256,
         epochs=20,
         validation_data=((X_val[:,:,:13] - MEAN) / STDEV, y_val),
         callbacks=[early_stopping]);

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


132/132 [==============================] - 246s 2s/step - loss: 3.0115 - accuracy: 0.0518 - val_loss: 3.0134 - val_accuracy: 0.0465
Epoch 2/20
132/132 [==============================] - 238s 2s/step - loss: 3.0002 - accuracy: 0.0508 - val_loss: 2.9959 - val_accuracy: 0.0522
Epoch 3/20
132/132 [==============================] - 237s 2s/step - loss: 2.9962 - accuracy: 0.0482 - val_loss: 2.9963 - val_accuracy: 0.0463
Epoch 4/20
132/132 [==============================] - 248s 2s/step - loss: 2.9960 - accuracy: 0.0516 - val_loss: 2.9962 - val_accuracy: 0.0517
Epoch 5/20
132/132 [==============================] - 238s 2s/step - loss: 2.9959 - accuracy: 0.0497 - val_loss: 2.9964 - val_accuracy: 0.0517
Epoch 6/20
132/132 [==============================] - 238s 2s/step - loss: 2.9959 - accuracy: 0.0498 - val_loss: 2.9965 - val_accuracy: 0.0517
Epoch 7/20
132/132 [==============================] - 246s 2s/step - loss: 2.9958 - accuracy: 0.0498 - val_loss: 2.9964 - val_accuracy: 0.0465


In [ ]:
learning_rate = 0.001

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

lstm.fit((X_train[:,:,:13] - MEAN) / STDEV, y_train,
         batch_size=256,
         epochs=20,
         validation_data=((X_val[:,:,:13] - MEAN) / STDEV, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 249s 2s/step - loss: 2.9956 - accuracy: 0.0505 - val_loss: 2.9965 - val_accuracy: 0.0465
Epoch 2/20
132/132 [==============================] - 247s 2s/step - loss: 2.9955 - accuracy: 0.0504 - val_loss: 2.9966 - val_accuracy: 0.0465
Epoch 3/20
132/132 [==============================] - 246s 2s/step - loss: 2.9955 - accuracy: 0.0504 - val_loss: 2.9967 - val_accuracy: 0.0517
Epoch 4/20
132/132 [==============================] - 246s 2s/step - loss: 2.9955 - accuracy: 0.0517 - val_loss: 2.9968 - val_accuracy: 0.0517
Epoch 5/20
 47/132 [=========>....................] - ETA: 2:25 - loss: 2.9951 - accuracy: 0.0515

KeyboardInterrupt: ignored

## Random Forest

In [27]:
rf = RandomForestClassifier(n_jobs=-1, verbose=1, min_impurity_decrease=.00001)
rf.fit((X_train[:,:,0] - np.mean(X_train[:,:,0])) / np.std(X_train[:,:,0]), y_train)

# rf.fit(X_train.reshape(X_train.shape[0], -1), y_train)
# lstm.fit((X_train[:,:,0] - np.mean(X_train[:,:,0])) / np.std(X_train[:,:,0]), y_train,
#          batch_size=256,
#          epochs=20,
#          validation_data=((X_val[:,:,0] - np.mean(X_train[:,:,0]) / np.std(X_train[:,:,0]), y_val),
#          callbacks=[early_stopping]);

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   23.4s finished


RandomForestClassifier(min_impurity_decrease=1e-05, n_jobs=-1, verbose=1)

In [28]:
y_pred_rf = rf.predict((X_val[:,:,0] - np.mean(X_train[:,:,0])) / np.std(X_train[:,:,0]))
print(classification_report(y_val, y_pred_rf))

              precision    recall  f1-score   support

           0       0.04      0.04      0.04       247
           1       0.06      0.07      0.06       226
           2       0.06      0.08      0.07       224
           3       0.06      0.06      0.06       261
           4       0.05      0.05      0.05       231
           5       0.07      0.09      0.08       233
           6       0.03      0.04      0.03       215
           7       0.03      0.02      0.02       235
           8       0.06      0.06      0.06       232
           9       0.06      0.06      0.06       234
          10       0.05      0.04      0.05       209
          11       0.05      0.06      0.05       235
          12       0.05      0.05      0.05       240
          13       0.05      0.05      0.05       241
          14       0.04      0.05      0.05       225
          15       0.04      0.04      0.04       225
          16       0.04      0.05      0.04       214
          17       0.05    

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.1s finished


## 1st MFCC

In [31]:
MEAN = X_train[:,:,0].mean()
STDEV = np.std(X_train[:,:,0])
MEAN, STDEV

(617.7811827258297, 176.7701128678243)

In [43]:
lstm = Sequential([Bidirectional(LSTM(64), input_shape=(32, 1)),
                  #  Bidirectional(LSTM(128, return_sequences=True)),
                  #  Bidirectional(LSTM(128)),
                  #  Dense(64, activation='sigmoid'),
                  #  Dropout(.5),
                   Dense(20, activation='softmax')]
                  )
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 128)              33792     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 20)                2580      
                                                                 
Total params: 36,372
Trainable params: 36,372
Non-trainable params: 0
_________________________________________________________________


In [44]:
lstm.input_shape, lstm.output_shape

((None, 32, 1), (None, 20))

In [48]:
early_stopping = EarlyStopping(monitor='loss', patience=4, min_delta=.001)
early_stopping_val = EarlyStopping(monitor='val_loss', patience=8, min_delta=.001)
callbacks = [early_stopping, early_stopping_val]

In [52]:
learning_rate = 0.01

lstm.compile(optimizer=Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

lstm.fit((X_train[:,:,0] - MEAN) / STDEV, y_train,
         batch_size=256,
         epochs=20,
         validation_data=((X_val[:,:,0] - MEAN) / STDEV, y_val),
         callbacks=[early_stopping]);

Epoch 1/20
132/132 [==============================] - 6s 12ms/step - loss: 2.9984 - accuracy: 0.0531 - val_loss: 2.9962 - val_accuracy: 0.0526
Epoch 2/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9972 - accuracy: 0.0500 - val_loss: 2.9969 - val_accuracy: 0.0496
Epoch 3/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9965 - accuracy: 0.0505 - val_loss: 2.9972 - val_accuracy: 0.0513
Epoch 4/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9963 - accuracy: 0.0530 - val_loss: 2.9982 - val_accuracy: 0.0483
Epoch 5/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9962 - accuracy: 0.0509 - val_loss: 2.9975 - val_accuracy: 0.0463
Epoch 6/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9960 - accuracy: 0.0525 - val_loss: 2.9974 - val_accuracy: 0.0520
Epoch 7/20
132/132 [==============================] - 1s 6ms/step - loss: 2.9961 - accuracy: 0.0529 - val_loss: 2.9971 - val_accuracy: 0.0546
Epoch

## Predict

In [ ]:
y_pred_lstm = lstm.predict(X_val[:, :, None])
print(classification_report(y_val, y_pred_lstm.argmax(axis=1)))